This code mostly is same as "First neighbor" but here I search not one nearest neighbor but all of them with equal distance and then search nearest across real size of them.

Also here will be comparing one part from train with rest four parts (validation) due to calculate the metric (in "First neighbor" test was compared with train).

So the result will save as sorted list of tuples like (dice coefficient, some image from first part, real nearest neighbor from other parts). Therefore you can look at the first few records in {result.csv} and find out the worst result and the pictures which gave it.

In [1]:
# imports and constants

import numpy as np # importing numpy for work with matrices
import csv # importing csv to work with train_masks.csv
from scipy.spatial import KDTree # importing KDTree for fast neighbor searching
from time import gmtime, strftime # importing some stuff from time to check the speed for algorithm

TIF = len(".tif")
EPS = 0.001

NUMBER_OF_PARTS = 5

SORTED_LIST = "input/sorted.csv"
ARCHIVE = "input/new_train_data.npz"
REAL_ARCHIVE = "input/train_data.npz"
RESULT = "input/result.csv"

In [2]:
# metric function
# x and y should be same-sized arrays of 0s and 1s

def dice_coefficient(x, y):
    if not(x.any() or y.any()):
        return 1
    return 2 * (x.astype(bool) * y.astype(bool)).sum() / (x.sum() + y.sum())

In [3]:
# forming %NUMBER_OF_PARTS% proportional parts of arrays in new_train (small images)

sort = []
with open(SORTED_LIST) as sorted_list:
    reader = csv.reader(sorted_list)
    for row in reader:
        if len(row) == 2:
            sort.append(row[1])

part = [i for i in range(NUMBER_OF_PARTS)]
for i in range(NUMBER_OF_PARTS):
    part[i] = sort[i::NUMBER_OF_PARTS]
    
del sort

In [4]:
print(strftime("%Y-%m-%d %H:%M:%S", gmtime())) # don't forget to add 3 hours

part_to_test = 0

test_names = part[part_to_test]
train_names = []
for i in range(NUMBER_OF_PARTS):
    if i != part_to_test:
        train_names += part[i]
        
del part_to_test

print(len(test_names))
print(len(train_names))
        
print(strftime("%Y-%m-%d %H:%M:%S", gmtime())) # don't forget to add 3 hours

2016-07-22 14:11:26
1127
4508
2016-07-22 14:11:26


In [5]:
print(strftime("%Y-%m-%d %H:%M:%S", gmtime())) # don't forget to add 3 hours

with np.load(ARCHIVE, mmap_mode='r') as data:
    test_images = np.array([data[image_name].flatten() for image_name in test_names]).astype(int)
    train_images = np.array([data[image_name].flatten() for image_name in train_names]).astype(int)
        
print(strftime("%Y-%m-%d %H:%M:%S", gmtime())) # don't forget to add 3 hours

2016-07-22 14:11:30
2016-07-22 14:12:46


In [12]:
print(strftime("%Y-%m-%d %H:%M:%S", gmtime())) # don't forget to add 3 hours

tree = KDTree(train_images, leafsize=200)
        
print(strftime("%Y-%m-%d %H:%M:%S", gmtime())) # don't forget to add 3 hours

2016-07-22 14:30:38
2016-07-22 14:33:02


In [17]:
print(strftime("%Y-%m-%d %H:%M:%S", gmtime())) # don't forget to add 3 hours

result = []

for i in range(len(test_images)):
    
    dist, ind = tree.query(test_images[i], k=1, p=2)
    neighbors_indeces = tree.query_ball_point(test_images[i], dist + EPS)

    with np.load(REAL_ARCHIVE, mmap_mode='r') as data:
        real_example = data[test_names[i]].flatten()
        real_neighbors = np.array([data[img].flatten() for img in np.array(train_names)[neighbors_indeces]]).astype(int)

    real_tree = KDTree(real_neighbors, leafsize=10)
    dist, ind = real_tree.query(real_example, k=1, p=2)
    real_neighbor_name = train_names[neighbors_indeces[ind]]

    with np.load(REAL_ARCHIVE, mmap_mode='r') as data:
        example_mask = data[test_names[i][:-TIF] + "_mask.tif"]
        neighbor_mask = data[real_neighbor_name[:-TIF] + "_mask.tif"]

    result.append((dice_coefficient(example_mask, neighbor_mask), test_names[i], real_neighbor_name))
    
result.sort()
    
print(strftime("%Y-%m-%d %H:%M:%S", gmtime())) # don't forget to add 3 hours

2016-07-22 16:04:26
2016-07-22 17:41:25


In [19]:
with open(RESULT, "w") as file:
    writer = csv.writer(file)
    writer.writerows(result)

In [24]:
result = np.array(result)
np.average(result[:,0].astype(float))

0.46927067909622322